In [1]:
%config Completer.use_jedi = False        # For autocomplete to work!

<H1 align="center">  🧸 Project: Toys 🧸 </H1>

---

In this notebook we'll build the scraper for our dataset. We're going to use *BeautifulSoup* (along with *requests8)) for the scraping and *sqlite3* to hold our data.

To follow my workflow, we'll take the following steps:

* 1️⃣ - Start by getting information on a single product from a serach result.
* 2️⃣ - Get information on all products on a single search result page and save it as a pandas dataframe.
* 3️⃣ - Get this information saved in a database using SQLite3.
* 4️⃣ - Build a script to get all pages from a search result.
* 5️⃣ - Loop through all pages and get information on all products saved in our database.
* 🎉 - Celebrate!

First of all, let's run our imports:

In [77]:
import requests
from bs4 import BeautifulSoup
import sqlite3 as sql
import pandas as pd
import time
import numpy as np

# 1️⃣ - Scraping item from search result

In [4]:
base_url = 'https://www.kohls.com'
girls_url = 'https://www.kohls.com/catalog/girls-toys.jsp?CN=Gender:Girls+Department:Toys'
boys_url = 'https://www.kohls.com/catalog/boys-toys.jsp?CN=Gender:Boys+Department:Toys'

In [4]:
headers = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

print('Requesting page...')
page = requests.get(girls_url, headers=headers, timeout=10)

print('Parsing content...')
soup1 = BeautifulSoup(page.content, "html.parser")

print('Prettifying...')
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

print('Getting product...')
product = soup2.find(class_='product-description')

# Link to product page
link = base_url + product.find(class_='prod_nameBlock').find('p')['rel']

# Product name
name = product.find(class_='prod_nameBlock').find('p').get_text().strip()

# Product price
try:
    price = product.find(class_='prod_price_amount').get_text().strip()[1:]
except:
    price = product.find(
        class_='prod_price_amount red_color').get_text().strip()[1:]

print(link)
print(name)
print(price)

Requesting page...
Parsing content...
Prettifying...
Getting product...
https://www.kohls.com/product/prd-4469399/barbie-3-in-1-dreamcamper-vehicle-and-accessories.jsp?prdPV=1
Barbie® 3-in-1 DreamCamper Vehicle and Accessories
89.99


# 2️⃣ - Getting all products in a search result page

In [5]:
print('Requesting page...')
page = requests.get(girls_url, headers=headers, timeout=10)

print('Parsing content...')
soup1 = BeautifulSoup(page.content, "html.parser")

print('Prettifying...')
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

print('Getting products...')
products = soup2.find_all(class_='product-description')

Requesting page...
Parsing content...
Prettifying...
Getting products...


In [6]:
len(products)

48

In [7]:
# Loop through all products and store info
data_dict = {
    'name':[],
    'price':[],
    'link':[]
}

for product in products:
    # Link to product page
    data_dict['link'].append(base_url + product.find(class_='prod_nameBlock').find('p')['rel'])

    # Product name
    data_dict['name'].append(product.find(class_='prod_nameBlock').find('p').get_text().strip())

    # Product price
    try:
        price = product.find(class_='prod_price_amount').get_text().strip()[1:]
    except:
        price = product.find(
            class_='prod_price_amount red_color').get_text().strip()[1:]
    data_dict['price'].append(price)

In [8]:
data_dict

{'name': ['Barbie® 3-in-1 DreamCamper Vehicle and Accessories',
  'Barbie® Dreamhouse Playset',
  'Step2 Modern Cook Play Kitchen Set',
  'Barbie® Dreamhouse Playset',
  'Hot Wheels Race Car & Track Play Table by Step2',
  'LEGO Harry Potter Hogwarts Chamber of Secrets 76389 Building Kit (1,176 Pieces)',
  'LEGO Marvel Infinity Gauntlet 76191 Building Kit (590 Pieces)',
  'LEGO Star Wars Advent Calendar 75307 Awesome Toy Building Kit for Kids (335 Pieces)',
  'LEGO Marvel The Avengers Advent Calendar 76196 Building Kit (298 Pieces)',
  'Disney Princess Jeep Wrangler Ride-On Vehicle by Fisher-Price Power Wheels',
  'Step2 Modern Cook Play Kitchen Pink Set',
  'LEGO Friends Advent Calendar 41690 Building Kit (370 Pieces)',
  'Epoch Games Super Mario Blow Up Shaky Tower Game',
  'Barbie® Club Chelsea Camper',
  'LEGO Harry Potter Hogsmeade Village Visit 76388 Building Kit (851 Pieces)',
  "Kohl's Cares Clifford the Big Red Dog Plush",
  'LEGO NINJAGO City Gardens 71741 LEGO Set (5,685 Pie

In [9]:
df = pd.DataFrame(data_dict)

In [10]:
df

,name,price,link
0,Barbie® 3-in-1 DreamCamper Vehicle and Accesso...,89.99,https://www.kohls.com/product/prd-4469399/barb...
1,Barbie® Dreamhouse Playset,199.99,https://www.kohls.com/product/prd-5115185/barb...
2,Step2 Modern Cook Play Kitchen Set,79.99,https://www.kohls.com/product/prd-1819866/step...
3,Barbie® Dreamhouse Playset,199.99,https://www.kohls.com/product/prd-5184120/barb...
4,Hot Wheels Race Car & Track Play Table by Step2,119.99,https://www.kohls.com/product/prd-3414834/hot-...
5,LEGO Harry Potter Hogwarts Chamber of Secrets ...,129.99,https://www.kohls.com/product/prd-5167485/lego...
6,LEGO Marvel Infinity Gauntlet 76191 Building K...,69.99,https://www.kohls.com/product/prd-5167443/lego...
7,LEGO Star Wars Advent Calendar 75307 Awesome T...,39.99,https://www.kohls.com/product/prd-5186969/lego...
8,LEGO Marvel The Avengers Advent Calendar 76196...,39.99,https://www.kohls.com/product/prd-5186971/lego...
9,Disney Princess Jeep Wrangler Ride-On Vehicle ...,299.99,https://www.kohls.com/product/prd-3284387/disn...


## Inside a product page - Getting product description

From product page:
```html
<div id="productDetailsTabContent" class="tabcontent" style="display: block; min-height: 222px; height: 222px;"><p> When kids hit the open road in the Barbie 3-in-1 DreamCamper, their stories can go wherever their imaginations lead. </p><br><p><a href=" target=" _blank"=""></a></p><br><p>PRODUCT FEATURES</p><ul> <li>When imaginations explore, they'll find multiple play areas including a living room for lounging, a patio with a pool, a bathroom with a pop-up shower, a rooftop bedroom and a kitchen with everything they need to play out cooking fun.</li> <li>With multiple transformations, 3 vehicles, 5 living spaces, 360-degree play and 50 accessories, the Barbie 3-in-1 DreamCamper encourages young minds to explore outdoor adventures and bring their travel dreams to life!</li> <li>When imaginations explore, they'll find multiple play areas including a living room for lounging, a patio with a pool, a bathroom with a pop-up shower, a rooftop bedroom and a kitchen with everything they need to play out cooking fun.</li> <li>Transformations add excitement to storytelling play -- unlatch the camper to reveal a boat and a pickup truck that seats 4; flip the coffee table to find a fire pit, and pop the camper's roof up to reveal a loft bed.</li> <li>Pack everything inside and close it up to hit the road again. A cool design and details -- like sweet stripes, Barbie logo details, silvery accents (on the grille and rolling wheels) -- make road tripping so much fun!</li> <li>With 50 play pieces, it's never the same adventure twice. Accessories include fishing gear, a picnic table with seating for 4, bathroom necessities, a sleeping bag, blankets and everything kids need to play out camping stories -- like s'mores!</li></ul><p>WHAT'S INCLUDED</p><ul> <li>Includes Barbie 3-in-1 DreamCamper vehicle, 1 detachable truck, 1 boat, 4 plates, 4 cups, 4 forks, 1 spatula, 1 cooking pan, milk, orange juice, 2 water bottles, 2 s'mores, 2 hot dogs, 1 fish, 4 toothbrushes, 1 toothbrush holder, 2 blankets, 1 sleeping bag, fishing pole, 2 stools, 1 transforming fire pit and 1 picnic table.</li></ul><p>PRODUCT DETAILS</p><ul> <li>Imported</li> <li>Age: 3 years &amp; up</li> <li>Resin</li> <li>4.48 X 8.77 X 3.38 in</li> <li>No warranty</li> <li>Spot clean</li></ul><p>MODEL NUMBERS</p><ul> <li>GHL93</li></ul></div>
```

We can get inside a single product url and look for its description. Let's test our idea on the first item in our dataframe:

In [11]:
product_url = df.iloc[0]['link']
print(product_url)

https://www.kohls.com/product/prd-4469399/barbie-3-in-1-dreamcamper-vehicle-and-accessories.jsp?prdPV=1


In [12]:
page = requests.get(product_url, headers=headers)

soup1 = BeautifulSoup(page.content, "html.parser")

soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

description = soup2.find('div',
                         class_='tabcontent').find('p').get_text().strip()

In [13]:
print(description)

When kids hit the open road in the Barbie 3-in-1 DreamCamper, their stories can go wherever their imaginations lead.


## Function to get info on product page

Ok, now that we are sure our method works, let's put it all inside a function to get the product description from its url:

In [7]:
def getProduct(URL):
    '''
    Given a URL from Kohl's products, return its description as a string.
    Uses the headers defined outside th function scope.
    '''
    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    
    try:
        description = soup2.find('div',
                             class_='tabcontent').find('p').get_text().strip()
    except:
        description = 'No info'
    
    return description

In [15]:
getProduct(product_url)

'When kids hit the open road in the Barbie 3-in-1 DreamCamper, their stories can go wherever their imaginations lead.'

## Let's try it out all out for the first page:

In [16]:
headers = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

print('Requesting page...')
page = requests.get(girls_url, headers=headers, timeout=10)

print('Parsing content...')
soup1 = BeautifulSoup(page.content, "html.parser")

print('Prettifying...')
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

print('Getting products...')
products = soup2.find_all(class_='product-description')

# Loop through all products and store info
data_dict = {
    'name':[],
    'price':[],
    'link':[],
    'description':[]
}

counter = 1
for product in products:
    print(f'Doing {counter} of {len(products)}')
    counter += 1
    
    # Link to product page
    link = base_url + product.find(class_='prod_nameBlock').find('p')['rel']
    data_dict['link'].append(link)

    # Product name
    data_dict['name'].append(product.find(class_='prod_nameBlock').find('p').get_text().strip())

    # Product price
    try:
        price = product.find(class_='prod_price_amount').get_text().strip()[1:]
    except:
        price = product.find(
            class_='prod_price_amount red_color').get_text().strip()[1:]
    data_dict['price'].append(price)
    
    # Product description
    data_dict['description'].append(getProduct(link))

Requesting page...
Parsing content...
Prettifying...
Getting products...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48


In [17]:
page1_df = pd.DataFrame(data_dict)

In [18]:
page1_df

,name,price,link,description
0,Barbie® 3-in-1 DreamCamper Vehicle and Accesso...,89.99,https://www.kohls.com/product/prd-4469399/barb...,When kids hit the open road in the Barbie 3-in...
1,Barbie® Dreamhouse Playset,199.99,https://www.kohls.com/product/prd-5115185/barb...,"Measuring 43 inches tall and 41 inches wide, t..."
2,Step2 Modern Cook Play Kitchen Set,79.99,https://www.kohls.com/product/prd-1819866/step...,Fire up your little one's love for cooking wit...
3,Barbie® Dreamhouse Playset,199.99,https://www.kohls.com/product/prd-5184120/barb...,The possibilities are endless for your creativ...
4,Hot Wheels Race Car & Track Play Table by Step2,119.99,https://www.kohls.com/product/prd-3414834/hot-...,Your little speedster will love all the racing...
5,LEGO Harry Potter Hogwarts Chamber of Secrets ...,129.99,https://www.kohls.com/product/prd-5167485/lego...,"The large, modular LEGO Harry Potter Hogwarts ..."
6,LEGO Marvel Infinity Gauntlet 76191 Building K...,69.99,https://www.kohls.com/product/prd-5167443/lego...,"The stunning, build-and-display LEGO Marvel In..."
7,LEGO Star Wars Advent Calendar 75307 Awesome T...,39.99,https://www.kohls.com/product/prd-5186969/lego...,Build excitement before Christmas with the 202...
8,LEGO Marvel The Avengers Advent Calendar 76196...,39.99,https://www.kohls.com/product/prd-5186971/lego...,Give Marvel fans a special Super Hero Christma...
9,Disney Princess Jeep Wrangler Ride-On Vehicle ...,299.99,https://www.kohls.com/product/prd-3284387/disn...,Your little princess can imagine heading off t...


# 3️⃣ - Get the first page on a database using SQLite3

Now let's save the data in a database (instead of a pandas dataframe). First off we need to actually create the database and connect to it:

In [19]:
## Start database for storing:
con = sql.connect('kohls_toys.db')
cur = con.cursor()

In [20]:
page1_df.columns

Index(['name', 'price', 'link', 'description'], dtype='object')

Creating a table, named `girlstoys` to store data from girls toys:

In [21]:
## Create table for girls toys
cur.execute('''CREATE TABLE IF NOT EXISTS girlstoys
                    (name text, price real, url text, description text)''')

Scraping:

In [22]:
headers = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

print('Requesting page...')
page = requests.get(girls_url, headers=headers, timeout=10)

print('Parsing content...')
soup1 = BeautifulSoup(page.content, "html.parser")

print('Prettifying...')
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

print('Getting products...')
products = soup2.find_all(class_='product-description')

# Loop through all products and store info
counter = 1
for product in products:
    print(f'Doing {counter} of {len(products)}')
    counter += 1
    
    # Link to product page
    link = base_url + product.find(class_='prod_nameBlock').find('p')['rel']    

    # Product name
    name = product.find(class_='prod_nameBlock').find('p').get_text().strip()

    # Product price
    try:
        price = product.find(class_='prod_price_amount').get_text().strip()[1:]
    except:
        price = product.find(
            class_='prod_price_amount red_color').get_text().strip()[1:]
    
    # Product description
    desc = getProduct(link)
    
    # This next line inserts the data into our sqlite database
    cur.execute('INSERT OR IGNORE INTO girlstoys VALUES (?, ?, ?, ?)',
                (name, price, link, desc))
    con.commit()

Requesting page...
Parsing content...
Prettifying...
Getting products...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48


## Getting a dataframe from SQLite database

With our database and table created, let's retrieve the data into a pandas dataframe to check if it all worked out:

In [23]:
# Simple query
page1_df_sql = pd.read_sql(f'SELECT * FROM girlstoys', con)

In [24]:
page1_df_sql

,name,price,url,description
0,Barbie® 3-in-1 DreamCamper Vehicle and Accesso...,89.99,https://www.kohls.com/product/prd-4469399/barb...,When kids hit the open road in the Barbie 3-in...
1,Barbie® Dreamhouse Playset,199.99,https://www.kohls.com/product/prd-5115185/barb...,"Measuring 43 inches tall and 41 inches wide, t..."
2,Step2 Modern Cook Play Kitchen Set,79.99,https://www.kohls.com/product/prd-1819866/step...,Fire up your little one's love for cooking wit...
3,Barbie® Dreamhouse Playset,199.99,https://www.kohls.com/product/prd-5184120/barb...,The possibilities are endless for your creativ...
4,Hot Wheels Race Car & Track Play Table by Step2,119.99,https://www.kohls.com/product/prd-3414834/hot-...,Your little speedster will love all the racing...
...,...,...,...,...
91,LEGO Technic Porsche 911 RSR 42096 LEGO Set,149.99,https://www.kohls.com/product/prd-3592353/lego...,Empower your child to get creative with this ...
92,Step2 Pump & Splash Discovery Pond Toy,149.99,https://www.kohls.com/product/prd-4961161/step...,Kids can create endless water-play fun with th...
93,VTech Drop & Go Dump Truck,17.99,https://www.kohls.com/product/prd-2625972/vtec...,The perfect partner for a day of learning fun....
94,Ceaco National Lampoon's Christmas Vacation 30...,9.99,https://www.kohls.com/product/prd-3018143/ceac...,Celebrate your favorite holiday movie with thi...


# 4️⃣ - Build a script to get all pages from a search result.

* Try to repeat the previous step and going to next page
---

## Creating a function to go to next page from search results

So, the website we are scraping has some dynamic features. We can use *Selenium* to find the **next page** button in the bottom of the search results page and click on it. Having done this, we can get the url of the next page and keep doing this until no **next page** button is found.

In [1]:
from selenium import webdriver

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait


driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get(
    'https://www.kohls.com/catalog/girls-toys.jsp?CN=Gender:Girls+Department:Toys&sks=true&kls_sbp=41944269349877692951351293429971195727&PPP=48&WS=4464&S=1'
)

# This next line waits for the page to render before finding elements
driver.implicitly_wait(10)

nxt = driver.find_element_by_xpath(
    "//*[@id='page-navigation-bottom']/div/a[2]")
driver.execute_script("arguments[0].click();", nxt)

print(driver.current_url)
driver.quit()



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
There is no [linux64] chromedriver for browser 95.0.4638 in cache
Get LATEST driver version for 95.0.4638
Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.54/chromedriver_linux64.zip
Driver has been saved in cache [/home/bruno/.wdm/drivers/chromedriver/linux64/95.0.4638.54]


https://www.kohls.com/catalog/girls-toys.jsp?CN=Gender:Girls+Department:Toys&sks=true&kls_sbp=41944269349877692951351293429971195727&PPP=48&WS=4464&S=1


Let's put this into a function to use in a loop:

In [5]:
def getNextPage(url, slp = 10):
    driver = webdriver.Chrome(ChromeDriverManager().install())

    driver.get(url)

    ##your code goes here##
    driver.implicitly_wait(slp)
    nxt = driver.find_element_by_xpath(
        "//*[@id='page-navigation-bottom']/div/a[2]")
    driver.execute_script("arguments[0].click();", nxt)
    next_url = driver.current_url
    driver.quit()
    if next_url != url:
        return next_url
    else:
        return

In [6]:
print(getNextPage(girls_url, 10))



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [/home/bruno/.wdm/drivers/chromedriver/linux64/95.0.4638.54/chromedriver] found in cache


https://www.kohls.com/catalog/girls-toys.jsp?CN=Gender:Girls+Department:Toys&PPP=48&WS=48&S=1&sks=true&kls_sbp=74625815023998607064484393812221555783


**Ran the above function in the script `getPages.py` to store all pages URLs in a text file named `girls_pages.txt`. Now, we can read the URLs and store them in a tuple.**

**Repeated the process for boys_pages.txt**

# 5️⃣ - Loop through all pages and get information on all products saved in our database
---

## Girls toys

In [85]:
with open('girls_pages.txt') as file:
    girls_pages = file.readlines()
    girls_pages = [line.rstrip() for line in girls_pages]

In [86]:
len(girls_pages)

94

In [87]:
girls_pages[:3]

['https://www.kohls.com/catalog/girls-toys.jsp?CN=Gender:Girls+Department:Toys',
 'https://www.kohls.com/catalog/girls-toys.jsp?CN=Gender:Girls+Department:Toys&PPP=48&WS=48&S=1&sks=true&kls_sbp=41944269349877692951351293429971195727',
 'https://www.kohls.com/catalog/girls-toys.jsp?CN=Gender:Girls+Department:Toys&sks=true&kls_sbp=41944269349877692951351293429971195727&PPP=48&WS=96&S=1']

In [88]:
## Start database for storing:
con = sql.connect('kohls_toys.db')
cur = con.cursor()

## Create table for girls toys
cur.execute('''CREATE TABLE IF NOT EXISTS girlstoys
                    (name text, price real, url text, description text)''')

headers = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}
current_page = 1

###### This is the url that needs to be looped with getNextPage
for girls_url in girls_pages:
    print(40 * '=')
    print(f'On page {current_page}')
    
    print(f'Requesting page {current_page}...')
    page = requests.get(girls_url, headers=headers, timeout=10)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    products = soup2.find_all(class_='product-description')

    # Loop through all products and store info
    counter = 1
    for product in products:
        print(f'Doing {counter} of {len(products)}')
        counter += 1

        # Link to product page
        link = base_url + product.find(class_='prod_nameBlock').find('p')['rel']    

        # Product name
        name = product.find(class_='prod_nameBlock').find('p').get_text().strip()

        # Product price
        try:
            price = product.find(class_='prod_price_amount').get_text().strip()[1:]
        except:
            price = product.find(
                class_='prod_price_amount red_color').get_text().strip()[1:]

        # Product description
        desc = getProduct(link)

        cur.execute('INSERT OR IGNORE INTO girlstoys VALUES (?, ?, ?, ?)',
                    (name, price, link, desc))
        con.commit()
        
    current_page += 1

con.close()

On page 1
Requesting page 1...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 2
Requesting page 2...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of

Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 12
Requesting page 12...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
D

Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 22
Requesting page 22...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 23
Requesting page 23...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 1

Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 33
Requesting page 33...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
D

Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 43
Requesting page 43...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 44
Requesting page 44...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Do

Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 54
Requesting page 54...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Do

Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 64
Requesting page 64...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
O

Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 75
Requesting page 75...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 

Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 85
Requesting page 85...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
D

---
## Boys toys

In [89]:
with open('boys_pages.txt') as file:
    boys_pages = file.readlines()
    boys_pages = [line.rstrip() for line in boys_pages]

In [90]:
len(boys_pages)

89

In [91]:
boys_pages[:3]

['https://www.kohls.com/catalog/boys-toys.jsp?CN=Gender:Boys+Department:Toys',
 'https://www.kohls.com/catalog/boys-toys.jsp?CN=Gender:Boys+Department:Toys&PPP=48&WS=48&S=1&sks=true&kls_sbp=41944269349877692951351293429971195727',
 'https://www.kohls.com/catalog/boys-toys.jsp?CN=Gender:Boys+Department:Toys&sks=true&kls_sbp=41944269349877692951351293429971195727&PPP=48&WS=96&S=1']

In [93]:
## Start database for storing:
con = sql.connect('kohls_toys.db')
cur = con.cursor()

## Create table for girls toys
cur.execute('''CREATE TABLE IF NOT EXISTS boystoys
                    (name text, price real, url text, description text)''')

headers = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}
current_page = 1

###### This is the url that needs to be looped with getNextPage
for boys_url in boys_pages:
    print(40 * '=')
    print(f'On page {current_page}')
    
    print(f'Requesting page {current_page}...')
    page = requests.get(boys_url, headers=headers, timeout=10)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    products = soup2.find_all(class_='product-description')

    # Loop through all products and store info
    counter = 1
    for product in products:
        print(f'Doing {counter} of {len(products)}')
        counter += 1

        # Link to product page
        link = base_url + product.find(class_='prod_nameBlock').find('p')['rel']    

        # Product name
        name = product.find(class_='prod_nameBlock').find('p').get_text().strip()

        # Product price
        try:
            price = product.find(class_='prod_price_amount').get_text().strip()[1:]
        except:
            price = product.find(
                class_='prod_price_amount red_color').get_text().strip()[1:]

        # Product description
        try:
            desc = getProduct(link)
        except:
            desc = 'No info'

        cur.execute('INSERT OR IGNORE INTO boystoys VALUES (?, ?, ?, ?)',
                    (name, price, link, desc))
        con.commit()
        
    current_page += 1

con.close()

On page 1
Requesting page 1...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 2
Requesting page 2...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of

Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 12
Requesting page 12...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
D

Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 22
Requesting page 22...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 23
Requesting page 23...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 1

Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 33
Requesting page 33...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
D

Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 43
Requesting page 43...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 44
Requesting page 44...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Do

Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 54
Requesting page 54...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Do

Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 64
Requesting page 64...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
O

Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 75
Requesting page 75...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 

Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
Doing 43 of 48
Doing 44 of 48
Doing 45 of 48
Doing 46 of 48
Doing 47 of 48
Doing 48 of 48
On page 85
Requesting page 85...
Doing 1 of 48
Doing 2 of 48
Doing 3 of 48
Doing 4 of 48
Doing 5 of 48
Doing 6 of 48
Doing 7 of 48
Doing 8 of 48
Doing 9 of 48
Doing 10 of 48
Doing 11 of 48
Doing 12 of 48
Doing 13 of 48
Doing 14 of 48
Doing 15 of 48
Doing 16 of 48
Doing 17 of 48
Doing 18 of 48
Doing 19 of 48
Doing 20 of 48
Doing 21 of 48
Doing 22 of 48
Doing 23 of 48
Doing 24 of 48
Doing 25 of 48
Doing 26 of 48
Doing 27 of 48
Doing 28 of 48
Doing 29 of 48
Doing 30 of 48
Doing 31 of 48
Doing 32 of 48
Doing 33 of 48
Doing 34 of 48
Doing 35 of 48
Doing 36 of 48
Doing 37 of 48
Doing 38 of 48
Doing 39 of 48
Doing 40 of 48
Doing 41 of 48
Doing 42 of 48
D

# 🎉 - Celebrate!

Yay!